In [1]:
!pip install tokenizers

In [2]:
import sentencepiece as spm
import pandas as pd
import numpy as np
import json
import time

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# 事前にモデルをダウンロードしておく
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = AutoModelForSequenceClassification.from_pretrained("elyza/ELYZA-japanese-Llama-2-7b", force_download=True, resume_download=False)
tokenizer = AutoTokenizer.from_pretrained("elyza/ELYZA-japanese-Llama-2-7b")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at elyza/ELYZA-japanese-Llama-2-7b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [6]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-14 02:57:50--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.005s  

2024-04-14 02:57:50 (200 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [8]:
# 以下は、モデルがダウンロード済みである前提で動作します。
# ステップ1: 前処理済みのコーパスを読み込む
corpus_file = "input.txt"

# ステップ2: 自作のトークナイザーを学習
vocab_size = 32000
model_prefix = "custom_tokenizer"

# SentencePieceモデルの学習
spm.SentencePieceTrainer.Train(
    f"--input={corpus_file} --model_prefix={model_prefix} --vocab_size={vocab_size} "
    "--character_coverage=0.9995 --model_type=bpe --max_sentence_length=4096"
)


In [10]:
# 学習済みモデルの読み込み
custom_tokenizer = spm.SentencePieceProcessor()
custom_tokenizer.Load(f"{model_prefix}.model")

# Llama Tokenizerの読み込み (事前ダウンロード済み)
llama_tokenizer = AutoTokenizer.from_pretrained("elyza/ELYZA-japanese-Llama-2-7b", local_files_only=True)
# Llama Tokenizerの読み込み
# llama_tokenizer = spm.SentencePieceProcessor()
# llama_tokenizer.Load("llama_tokenizer.model")


# 評価用のテキストデータを読み込む (.jsonl形式)
def load_jsonl(file_path):
    data = []
    with open(file_path, "r") as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

In [11]:
from google.colab import drive
drive.mount('/content/drive')

# ファイルパスを指定して読み込む
eval_data = load_jsonl('/content/drive/MyDrive/en_med_sample.jsonl')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import transformers
print(f"transformers version: {transformers.__version__}")

import tokenizers
print(f"tokenizers version: {tokenizers.__version__}")

print(type(custom_tokenizer))
print(type(llama_tokenizer))

transformers version: 4.38.2
tokenizers version: 0.15.2
<class 'sentencepiece.SentencePieceProcessor'>
<class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>


In [13]:
import time
import numpy as np
from typing import Union
from transformers import LlamaTokenizerFast
import sentencepiece
# ステップ3: 評価コードの作成

def evaluate_tokenizer(text, tokenizer: Union[LlamaTokenizerFast, sentencepiece.SentencePieceProcessor]):
    start_time = time.time()
    if isinstance(tokenizer, LlamaTokenizerFast):
        tokens = tokenizer.encode(text, add_special_tokens=False)
    else:
        tokens = tokenizer.encode(text)
    end_time = time.time()
    elapsed_time = end_time - start_time

    if isinstance(tokenizer, LlamaTokenizerFast):
        unk_token_id = tokenizer.unk_token_id
        unk_ratio = len([t for t in tokens if t == unk_token_id]) / len(tokens)
    else:
        unk_id = tokenizer.unk_id()
        unk_ratio = len([t for t in tokens if t == unk_id]) / len(tokens)

    avg_length = len(tokens)

    return tokens, unk_ratio, avg_length, elapsed_time


In [14]:
import numpy as np

results = []
for text in eval_data["text"]:
    custom_tokens, custom_unk_ratio, custom_avg_length, custom_time = evaluate_tokenizer(text, custom_tokenizer)
    llama_tokens, llama_unk_ratio, llama_avg_length, llama_time = evaluate_tokenizer(text, llama_tokenizer)

    # 0除算を防ぐ
    if custom_tokens == 0 and llama_tokens == 0:
        custom_perplexity = 100.0
        llama_perplexity = 100.0
    elif custom_tokens == 0:
        custom_perplexity = 100.0
        llama_perplexity = float(np.exp(-np.log(llama_tokens) / llama_avg_length).mean())
    elif llama_tokens == 0:
        custom_perplexity = float(np.exp(-np.log(custom_tokens) / custom_avg_length).mean())
        llama_perplexity = 100.0
    else:
        custom_perplexity = float(np.exp(-np.log(custom_tokens) / custom_avg_length).mean())
        llama_perplexity = float(np.exp(-np.log(llama_tokens) / llama_avg_length).mean())

    # print(f"custom_tokens: {custom_tokens}, custom_avg_length: {custom_avg_length}, custom_perplexity: {custom_perplexity}")
    # print(f"llama_tokens: {llama_tokens}, llama_avg_length: {llama_avg_length}, llama_perplexity: {llama_perplexity}")

    results.append({
        "input": text,
        "custom_perplexity": custom_perplexity,
        "custom_time": custom_time,
        "custom_tokens": custom_tokens,
        "custom_unk_ratio": custom_unk_ratio,
        "custom_avg_length": custom_avg_length,
        "llama_perplexity": llama_perplexity,
        "llama_time": llama_time,
        "llama_tokens": llama_tokens,
        "llama_unk_ratio": llama_unk_ratio,
        "llama_avg_length": llama_avg_length
    })

# 結果を表示
print("Input | Custom Perplexity | Custom Time | Llama Perplexity | Llama Time")
for res in results:
    print(f"{res['input'][:50]:<50} | {res['custom_perplexity']:<17.2f} | {res['custom_time']:<10.2f} | "
          f"{res['llama_perplexity']:<16.2f} | {res['llama_time']:<9.2f}")

<ipython-input-14-49874f97bbcd>:19: RuntimeWarning: divide by zero encountered in log
  custom_perplexity = float(np.exp(-np.log(custom_tokens) / custom_avg_length).mean())


Streaming output truncated to the last 5000 lines.
glenmuise 4 months ago
rOKERFELLER MEDICINE IS TAK | inf               | 0.00       | 0.97             | 0.00     
Hans-Joachim Brucherseifer - Product Trailer
› 4K- | inf               | 0.01       | 1.00             | 0.01     
Upc general conference 2019
I intend, now, to turn | inf               | 0.03       | 1.00             | 0.02     
Psychiatric hospital for jail inmates inaugurated- | inf               | 0.00       | 0.99             | 0.00     
An exploratory study of engagement in a technology | inf               | 0.05       | 1.00             | 0.04     
Medical marijuana and the condition it treats – Pi | inf               | 0.01       | 1.00             | 0.00     
This report, produced by the Resuscitation Researc | inf               | 0.00       | 0.96             | 0.00     
One Health Poultry Hub for developing countries la | inf               | 0.00       | 0.99             | 0.00     
The Emory Critical Care APP F

In [21]:
example_input = "今日は晴れです。私は公園に行きました。"
custom_tokens, custom_unk_ratio, custom_avg_length, custom_time = evaluate_tokenizer(example_input, custom_tokenizer)
llama_tokens, llama_unk_ratio, llama_avg_length, llama_time = evaluate_tokenizer(example_input, llama_tokenizer)

print(f"Input: {example_input}")

print(f"custom_tokens: {custom_tokens}, type(custom_tokens): {type(custom_tokens)}")
print(f"custom_avg_length: {custom_avg_length}, type(custom_avg_length): {type(custom_avg_length)}")

if isinstance(custom_tokens, list):
    custom_perplexities = []
    for token, avg_len in zip(custom_tokens, [custom_avg_length] * len(custom_tokens)):
        if token == 0:
            custom_perplexities.append(100.0)
        else:
            custom_perplexities.append(float(np.exp(-np.log(token) / avg_len)))
    custom_perplexity = np.mean(custom_perplexities)
    print(f"custom_perplexities: {custom_perplexities}")
else:
    if custom_tokens == 0:
        custom_perplexity = 100.0
    else:
        custom_perplexity = float(np.exp(-np.log(custom_tokens) / custom_avg_length))

print(f"llama_tokens: {llama_tokens}, type(llama_tokens): {type(llama_tokens)}")
print(f"llama_avg_length: {llama_avg_length}, type(llama_avg_length): {type(llama_avg_length)}")

if isinstance(llama_tokens, list):
    llama_perplexities = []
    for token, avg_len in zip(llama_tokens, [llama_avg_length] * len(llama_tokens)):
        if token == 0:
            llama_perplexities.append(100.0)
        else:
            llama_perplexities.append(float(np.exp(-np.log(token) / avg_len)))
    llama_perplexity = np.mean(llama_perplexities)
    print(f"llama_perplexities: {llama_perplexities}")
else:
    if llama_tokens == 0:
        llama_perplexity = 100.0
    else:
        llama_perplexity = float(np.exp(-np.log(llama_tokens) / llama_avg_length))

print(f"Custom Perplexity: {custom_perplexity:.2f}")
print(f"Custom Time: {custom_time:.2f} sec")
print(f"Llama Perplexity: {llama_perplexity:.2f}")
print(f"Llama Time: {llama_time:.2f} sec")

Input: 今日は晴れです。私は公園に行きました。
custom_tokens: [31941, 0], type(custom_tokens): <class 'list'>
custom_avg_length: 2, type(custom_avg_length): <class 'int'>
custom_perplexities: [0.005595330518871602, 100.0]
llama_tokens: [29871, 31482, 30325, 30449, 233, 156, 183, 30553, 30499, 30427, 30267, 234, 170, 132, 30449, 30539, 31179, 30353, 30448, 30538, 30441, 30326, 30366, 30267], type(llama_tokens): <class 'list'>
llama_avg_length: 24, type(llama_avg_length): <class 'int'>
llama_perplexities: [0.6509254618789976, 0.6495023651585786, 0.6505164744164805, 0.6504058770975537, 0.7968198967267544, 0.8102514341370843, 0.8048801271414969, 0.6503134792090352, 0.6503614140722948, 0.6504254649282589, 0.650568367229492, 0.796677721490357, 0.8073551655312357, 0.8159109334919402, 0.6504058770975537, 0.6503258982870945, 0.6497641446915604, 0.6504914597044373, 0.6504067671336285, 0.6503267855910195, 0.6504129982388622, 0.650515580619693, 0.6504798539005602, 0.650568367229492]
Custom Perplexity: 50.00
Custom Ti